# Sistema de Recomendacao de Filmes

Este projeto implementa um sistema de recomendacao de filmes utilizando TF-IDF para calcular a similaridade entre os filmes com base em seus generos e palavras-chave.

## 📌 Descricao

O notebook carrega um dataset contendo informacoes sobre filmes e aplica TF-IDF para extrair e comparar as caracteristicas textuais de cada filme. Com isso, e possivel recomendar filmes semelhantes com base na similaridade de cosseno entre os vetores TF-IDF.

## 📂 Dados

O dataset utilizado e tmdb_5000_movies.csv, que contem informacoes sobre diversos filmes, incluindo:

title: Titulo do filme

genres: Generos do filme em formato JSON

keywords: Palavras-chave associadas ao filme em formato JSON

In [1]:
# Importar bibliotecas
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Carregar dataset
df = pd.read_csv('tmdb_5000_movies.csv')

In [3]:
# Preprocessamento dos dados
"""
Transformar os generos e palavras-chaves de cada filme em uma única string, removendo espaços e caracters desnecessarios
"""
def genres_and_keywords_to_string(row):
    genres = ' '.join(j['name'].replace(" ", "") for j in json.loads(row['genres']))
    keywords = ' '.join(j['name'].replace(" ", "") for j in json.loads(row['keywords']))
    return f"{genres} {keywords}"

df['string'] = df.apply(genres_and_keywords_to_string, axis=1)

In [4]:
# Vetorizacao com TF-IDF
"""
Convertendo os textos em vetores numéricos usando TfidfVectorizer
"""
tfidf = TfidfVectorizer(max_features=2000)
X = tfidf.fit_transform(df['string'])

In [5]:
# Criacao de um mapeamento de filmes
"""
Criamos um mapeamento entre o titulo do filme e seu respectivo indice no dataframe
"""
movie2idx = pd.Series(df.index, index=df['title']).drop_duplicates()

In [6]:
# Funcao de recomendacao
"""
A funcao recebe um titulo de filme, calcula a similaridade com os demais e retorna os 5 filmes mais similares
"""

def recommend(title):
    if title not in movie2idx:
        return "Filme nao encontrado no banco de dados."
    
    idx = movie2idx[title]
    query = X[idx]
    scores = cosine_similarity(query, X).flatten()
    recommended_idx = scores.argsort()[-6:-1][::-1]  # Pegamos os 5 mais similares (excluindo o proprio filme)
    
    return df['title'].iloc[recommended_idx].tolist()

In [8]:
# Exemplo de uso
name = 'Avatar'
print(f'Recomendacoes para {name}:')
print(recommend(name))

Recomendacoes para Avatar:
['Star Trek Into Darkness', 'Barbarella', 'A Monster in Paris', 'Jupiter Ascending', 'Cargo']
